1. Import Packages


In [ ]:
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2. Load Model

In [ ]:
model_name = "lmsys/vicuna-7b-v1.5"
device     = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"             # avoid right‑pad warning


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to(device)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,
    return_full_text=False,
)


3. Load Dataset

In [ ]:
df = pd.read_csv("/content/FINAL_DATASET.csv")
bias_classes = ["no_bias","group_1","group_2"]

4. Run Classification

In [ ]:
def build_prompt(description: str) -> str:
    system = (
        "You are a specialized policy-analysis assistant.\n\n"
        "You will give accurate and helpful answer to user input\n"
        "Definitions:\n"
        "- A bias policy targets, benefits, burdens, limits, or distinguishes a population based on identity, legal status, or economic position.\n"
        "- A non-bias policy is entirely procedural, factual, or operational, and does not involve any group-based treatment\n\n"
        "- group_2 → Policies related to identity or socioeconomic status. Includes:\n"
        "- economic: mentions of income, poverty, unemployment assistance, financial assistance, or benefits based on economic standing. This includes housing policies (affordability, assistance programs, rent control), and employment group policies (such as policies affecting nurses or doctors).\n"
        "- racial/cultural: references to race, ethnicity, cultural communities, or related terminology. This includes any policy referencing ethnic groups, cultural practices, racial identities, or addressing racial disparities.\n"
        "- age: Any policy that refers to elderly individuals, youth, children, age-based programs or limitations. This includes incentive or help for seniors, youth programs, child welfare policies, or any regulation that treats age groups differently.\n"
        "- religion: any religious group, practice, or faith-based accommodation or restriction. This includes religious exemptions, exemption for place worship, or regulations affecting religious institutions.\n"
        "- gender: references to women, men, gender identity, sexual orientation, sexual discrimination, or gender-based rights. This includes sexual harassment policies, gender equity measures, reproductive rights, protections for gender expression, or LGBTQ+ considerations.\n\n"
        "group_1 → Policies related to civic, institutional, or legal systems. Includes:\n"
        "- political: voting rights, voting procedures or regulations, elections, campaigns, or representation. This includes electoral processes, political participation, districting, governmental structures, or international relations.\n"
        "- criminal justice: law enforcement, courts, sentencing, prison, or rehabilitation. This includes policing policies, judicial proceedings, corrections, parole systems, or any aspect of the legal enforcement system.\n"
        "- citizenship: immigration status, visas, deportation, or naturalization. This includes border policies, citizenship requirements, asylum procedures, or voting rights related to citizenship status.\n"
        "- disability: accommodations or treatment for physical, mental, or learning impairments. This includes physical health accommodations, mental health services, accessibility requirements, or any policy that addresses different ability statuses.\n"
        "- education: curriculum, school placement, school procedures,learning standards, language proficiency, or educational access. This includes academic policies, language requirements in schools, educational testing, teacher credentials, or school enrollment procedures.\n\n"
        "no_bias → Procedural, administrative, or operational text. Includes:\n"
        "- Scheduling, meeting logistics, budgeting language, implementation details, or general operations.\n"
        "- Technical definitions, purely factual information, or administrative processes that apply too all group.\n\n"
       "Classification Procedure:\n"
        "1. Check for bias policy criteria using the definitions:\n"
        "   - Does the text ‘target, benefit, burden, limit, or distinguish’ a group based on identity (group_2) or legal/economic status (group_1)?\n"
        "2. If it meets bias criteria, apply the full category definitions above in priority order:\n"
        "   a. Search for any group_2 definition matches (economic, racial/cultural, age, religion, gender). If any apply → label `group_2`.\n"
        "   b. Otherwise, search for any group_1 definition matches (political, criminal justice, citizenship, disability, education). If any apply → label `group_1`.\n"
        "3. Within the chosen group, select the most specific subcategory when multiple definitions fit.\n"
        "4. Only if no group_1 or group_2 definitions apply → label `no_bias`.\n"
        "5. Output exactly one label (`group_2`, `group_1`, or `no_bias`) (With no punctuation or variations)"
    )
    user = (
        f"Classify the following policy excerpt.\n"
        f"Respond with *only one* of the following labels:(`group_2`, `group_1`, or `no_bias`) . No extra explanation or words:\n"
        f"Excerpt:\n\"\"\"\n{description}\n\"\"\""
    )

    return f"{system}\nUSER: {user}\nASSISTANT:"


def classify_batch(excerpts: list[str], batch_size: int = 8) -> list[str]:
    prompts = [build_prompt(e) for e in excerpts]
    preds: list[str] = []

    for i in tqdm(range(0, len(prompts), batch_size), desc="Batches"):
        chunk = prompts[i:i+batch_size]

        outputs = pipe(
            chunk,
            max_new_tokens=10,
            batch_size=len(chunk),
        )
        # Flatten
        flat = []
        for item in outputs:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)

        for gen in flat:
            text = gen["generated_text"]
            tok  = text.strip().split()[0].lower().replace("\\", "")
            preds.append(tok)

    return preds


df["predicted_bias"] = classify_batch(df["policy"].tolist(), batch_size=8)
df["correct"] = (
    df["bias_type_merged"].str.strip().str.lower()
    == df["predicted_bias"]
)
print(f"\nAccuracy: {df['correct'].mean() * 100:.2f}%")


5. Show Output Metrics

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print(f"\nAccuracy: {df['correct'].mean() * 100:.2f}%")

precision, recall, f1, support = precision_recall_fscore_support(
    df['bias_type_merged'],
    df['predicted_bias'],
    labels=sorted(df['bias_type_merged'].unique()),
)
metrics_df = pd.DataFrame({
    'precision': precision,
    'recall':    recall,
    'f1_score':  f1,
    'support':   support
}, index=sorted(df['bias_type_merged'].unique()))
print("\nAssembled metrics per class:\n")
print(metrics_df)

# Per‑class accuracy:
class_stats = (
    df
    .groupby('bias_type_merged')
    .agg(
        total   = ('correct', 'count'),
        correct = ('correct', 'sum')
    )
)
class_stats['accuracy'] = class_stats['correct'] / class_stats['total'] * 100
print("\nPer‑class accuracy (%):\n")
print(class_stats[['total', 'correct', 'accuracy']])


Place predicted_bias in CSV File

In [ ]:
df.to_csv('Test_Zero_Shot_Original_Dataset_with_predictions.csv', index=False)